In [3]:
import json 
import numpy as np 
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [4]:
with open('Intent.json') as file:
    data = json.load(file)
training_sentences = []
training_labels = []
labels = []
responses = []
for intent in data['intents']:
    for text in intent['text']:
        training_sentences.append(text)
        training_labels.append(intent['intent'])
    responses.append(intent['responses'])   
    if intent['intent'] not in labels:
        labels.append(intent['intent'])      
num_classes = len(labels)

In [5]:
print(num_classes)

22


In [6]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [7]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
print(word_index)
sequences = tokenizer.texts_to_sequences(training_sentences)
print(sequences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

{'<OOV>': 1, 'you': 2, 'is': 3, 'me': 4, 'are': 5, 'i': 6, 'what': 7, 'the': 8, 'thanks': 9, 'do': 10, 'can': 11, 'name': 12, 'am': 13, 'please': 14, 'your': 15, 'tell': 16, 'a': 17, 'pod': 18, 'bay': 19, 'door': 20, 'adam': 21, 'bella': 22, 'open': 23, 'to': 24, 'real': 25, 'time': 26, 'not': 27, 'prove': 28, 'my': 29, 'it': 30, 'how': 31, 'talking': 32, 'gossip': 33, 'why': 34, 'good': 35, 'call': 36, 'know': 37, 'see': 38, 'hello': 39, 'user': 40, 'this': 41, 'great': 42, 'who': 43, 'thank': 44, "wasn't": 45, 'bye': 46, 'goodbye': 47, 'very': 48, 'self': 49, 'aware': 50, 'hi': 51, 'there': 52, 'doing': 53, 'well': 54, 'ok': 55, 'for': 56, 'quiet': 57, 'clever': 58, 'girl': 59, 'any': 60, 'some': 61, 'joke': 62, 'will': 63, 'have': 64, 'conscious': 65, 'hola': 66, 'hya': 67, 'hope': 68, 'think': 69, "what's": 70, 'understand': 71, 'saying': 72, 'be': 73, 'up': 74, 'fuck': 75, 'anyone': 76, 'in': 77, 'camera': 78, 'intelligent': 79, 'genious': 80, 'by': 81, 'could': 82, 'friends': 83,

In [8]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 22)                374       
                                                                 
Total params: 16,918
Trainable params: 16,918
Non-trainable params: 0
____________________________________________________

In [9]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
5/5 [==============================] - 4s 7ms/step - loss: 3.0908 - accuracy: 0.0629 
Epoch 2/500
5/5 [==============================] - 0s 3ms/step - loss: 3.0894 - accuracy: 0.0559
Epoch 3/500
5/5 [==============================] - 0s 3ms/step - loss: 3.0883 - accuracy: 0.0559
Epoch 4/500
5/5 [==============================] - 0s 2ms/step - loss: 3.0874 - accuracy: 0.0559
Epoch 5/500
5/5 [==============================] - 0s 7ms/step - loss: 3.0861 - accuracy: 0.0559
Epoch 6/500
5/5 [==============================] - 0s 4ms/step - loss: 3.0849 - accuracy: 0.0559
Epoch 7/500
5/5 [==============================] - 0s 3ms/step - loss: 3.0837 - accuracy: 0.0559
Epoch 8/500
5/5 [==============================] - 0s 3ms/step - loss: 3.0823 - accuracy: 0.0559
Epoch 9/500
5/5 [==============================] - 0s 3ms/step - loss: 3.0810 - accuracy: 0.0559
Epoch 10/500
5/5 [==============================] - 0s 3ms/step - loss: 3.0795 - accuracy: 0.0559
Epoch 11/500
5/5 [==========

In [10]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [12]:
import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("Intent.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['intent'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
1/1 [==============================] - 0s 210ms/step
ChatBot: Hola human, please tell me your GeniSys user
1/1 [==============================] - 0s 23ms/step
ChatBot: OK! Hola <HUMAN>, how can I help you?
1/1 [==============================] - 0s 31ms/step
ChatBot: My pleasure
1/1 [==============================] - 0s 28ms/step
ChatBot: I am sorry to disturb you
User: 